<a href="https://colab.research.google.com/github/manoj7pal/Airline-Analysis-Project-via-Apache-Hive/blob/master/Analyzing_Police_Activity_with_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Police Activity with pandas

In [86]:
import pandas as pd

## 1. Load the Data

In [87]:
# Datasets

police_stops_ri = pd.read_csv('https://assets.datacamp.com/production/repositories/1497/datasets/62bd9feef451860db02d26553613a299721882e8/police.csv', parse_dates = True)
weather_rhode = pd.read_csv('https://assets.datacamp.com/production/repositories/1497/datasets/02f3fb2d4416d3f6626e1117688e0386784e8e55/weather.csv', parse_dates = True)

police_stops_ri.shape, weather_rhode.shape

((91741, 15), (4017, 27))

In [88]:
police_stops_ri.head()

,state,stop_date,stop_time,county_name,driver_gender,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district
0,RI,2005-01-04,12:55,NaN,M,White,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False,Zone X4
1,RI,2005-01-23,23:15,NaN,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone K3
2,RI,2005-02-17,04:15,NaN,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X4
3,RI,2005-02-20,17:15,NaN,M,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False,Zone X1
4,RI,2005-02-24,01:20,NaN,F,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X3


In [89]:
weather_rhode.head()

,STATION,DATE,TAVG,TMIN,TMAX,AWND,WSF2,WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22
0,USW00014765,2005-01-01,44.0,35,53,8.95,25.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00014765,2005-01-02,36.0,28,44,9.40,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
2,USW00014765,2005-01-03,49.0,44,53,6.93,17.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,USW00014765,2005-01-04,42.0,39,45,6.93,16.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,USW00014765,2005-01-05,36.0,28,43,7.83,17.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN


## 2. Preparing the Data
  a. Examine the Data.<br/> 
  b. Clean the Data. 

In [90]:
# Locate missing Values
print(police_stops_ri.isna().sum())
# print(weather_rhode.isna().sum())

state                     0
stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5205
driver_race            5202
violation_raw          5202
violation              5202
search_conducted          0
search_type           88434
stop_outcome           5202
is_arrested            5202
stop_duration          5202
drugs_related_stop        0
district                  0
dtype: int64


In [91]:
""" 
Drop irrelevant columns
  - County name: Missing Values
  - State: Dataset is of RI only
  - search_type: Lots of missing values 
"""
print(police_stops_ri.shape)
police_stops_ri.drop(labels = ['state', 'county_name', 'search_type'], axis = 'columns', inplace=True)
print(police_stops_ri.shape)

(91741, 15)
(91741, 12)


In [92]:
"""
The driver_gender column will be critical to many of your analyses. Because only a small fraction of rows are missing driver_gender, we'll drop those rows from the dataset.
"""

police_stops_ri.dropna(subset = ['driver_gender'], axis= 'rows', inplace=True)
print(police_stops_ri.shape)
print(police_stops_ri.isna().sum())

(86536, 12)
stop_date             0
stop_time             0
driver_gender         0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
district              0
dtype: int64


In [93]:
print(police_stops_ri.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86536 entries, 0 to 91740
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           86536 non-null  object
 1   stop_time           86536 non-null  object
 2   driver_gender       86536 non-null  object
 3   driver_race         86536 non-null  object
 4   violation_raw       86536 non-null  object
 5   violation           86536 non-null  object
 6   search_conducted    86536 non-null  bool  
 7   stop_outcome        86536 non-null  object
 8   is_arrested         86536 non-null  object
 9   stop_duration       86536 non-null  object
 10  drugs_related_stop  86536 non-null  bool  
 11  district            86536 non-null  object
dtypes: bool(2), object(10)
memory usage: 7.4+ MB
None


In [94]:
# Fixing Data Types
  # Stop Date and time to Date time
  # is-arrested  to bool

# police_stops_ri['stop_date'] = pd.to_datetime(police_stops_ri.stop_date)
# police_stops_ri['stop_time'] = pd.to_datetime(police_stops_ri.stop_time)

police_stops_ri['is_arrested'] = police_stops_ri.is_arrested.astype('bool')

print(police_stops_ri.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86536 entries, 0 to 91740
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           86536 non-null  object
 1   stop_time           86536 non-null  object
 2   driver_gender       86536 non-null  object
 3   driver_race         86536 non-null  object
 4   violation_raw       86536 non-null  object
 5   violation           86536 non-null  object
 6   search_conducted    86536 non-null  bool  
 7   stop_outcome        86536 non-null  object
 8   is_arrested         86536 non-null  bool  
 9   stop_duration       86536 non-null  object
 10  drugs_related_stop  86536 non-null  bool  
 11  district            86536 non-null  object
dtypes: bool(3), object(9)
memory usage: 6.8+ MB
None


In [95]:
# Combine stop_date and time - Convert to datetime format - Creating a DateTimeIndex 

police_stops_ri['stop_date_time'] = police_stops_ri.stop_date.str.cat(police_stops_ri.stop_time, sep= ' ')
police_stops_ri['stop_date_time'] = pd.to_datetime(police_stops_ri['stop_date_time'])
police_stops_ri.set_index('stop_date_time', inplace = True)

police_stops_ri.drop(labels = ['stop_date', 'stop_time'], axis = 'columns', inplace=True)
police_stops_ri.head()

,driver_gender,driver_race,violation_raw,violation,search_conducted,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district
stop_date_time,,,,,,,,,,
2005-01-04 12:55:00,M,White,Equipment/Inspection Violation,Equipment,False,Citation,False,0-15 Min,False,Zone X4
2005-01-23 23:15:00,M,White,Speeding,Speeding,False,Citation,False,0-15 Min,False,Zone K3
2005-02-17 04:15:00,M,White,Speeding,Speeding,False,Citation,False,0-15 Min,False,Zone X4
2005-02-20 17:15:00,M,White,Call for Service,Other,False,Arrest Driver,True,16-30 Min,False,Zone X1
2005-02-24 01:20:00,F,White,Speeding,Speeding,False,Citation,False,0-15 Min,False,Zone X3


## 3. Analysis
  **- Does the genders affects the Speeding Violation - Outcomes?**


In [164]:
# Unique Values: Suited for Categorical values
print(police_stops_ri.violation.value_counts())

Speeding               48423
Moving violation       16224
Equipment              10921
Other                   4409
Registration/plates     3703
Seat belt               2856
Name: violation, dtype: int64


In [165]:
print(police_stops_ri.violation.value_counts(normalize=True) * 100)

Speeding               55.957058
Moving violation       18.748267
Equipment              12.620181
Other                   5.094989
Registration/plates     4.279144
Seat belt               3.300361
Name: violation, dtype: float64


In [116]:
# Proportion or Percentages of Values of a column
print(police_stops_ri.stop_outcome.value_counts(normalize=True) * 100)

Citation            89.085467
Warning              5.935102
Arrest Driver        3.160534
No Action            0.721087
N/D                  0.701442
Arrest Passenger     0.396367
Name: stop_outcome, dtype: float64


In [179]:
# Speeding and Gender Wise - Outcome Proportion
male_speeding = police_stops_ri[ (police_stops_ri.driver_gender == 'M') & (police_stops_ri.violation == 'Speeding') ]
female_speeding = police_stops_ri[ (police_stops_ri.driver_gender == 'F') & (police_stops_ri.violation == 'Speeding') ]

all = pd.DataFrame()
all['Male %'] = male_violations.stop_outcome.value_counts(normalize=True) * 100
all['Female %'] = female_violations.stop_outcome.value_counts(normalize=True) * 100

print(all.head())

                  Male %   Female %
Citation       88.971033  89.387566
Warning         5.817214   6.246320
Arrest Driver   3.471846   2.338689
N/D             0.699468   0.706654
No Action       0.673975   0.845461


**- Does the genders commit different violations?**

In [185]:
# Gender Wise Violations %
male = police_stops_ri[(police_stops_ri.driver_gender == 'M')]
female = police_stops_ri[(police_stops_ri.driver_gender == 'F')]

all = pd.DataFrame()
all['Male %'] = male.violation.value_counts(normalize=True) * 100
all['Female %'] = female.violation.value_counts(normalize=True) * 100
print(all)

                        Male %   Female %
Speeding             52.224276  65.811391
Moving violation     20.614385  13.821822
Equipment            13.415761  10.519896
Other                 5.898474   2.973837
Registration/plates   4.217520   4.441827
Seat belt             3.629585   2.431227


**- Does the genders affects Arrest - for different violations?**

In [183]:
# Arrest % - Gender Wise
male_arrest = police_stops_ri[(police_stops_ri.driver_gender == 'M') & (police_stops_ri.is_arrested == True)]
female_arrest = police_stops_ri[(police_stops_ri.driver_gender == 'F') & (police_stops_ri.is_arrested == True)]

all = pd.DataFrame()
all['Male %'] = male_arrest.violation.value_counts(normalize=True) * 100
all['Female %'] = female_arrest.violation.value_counts(normalize=True) * 100
print(all)

                        Male %   Female %
Moving violation     30.136986  31.988042
Speeding             23.370693  14.947683
Equipment            22.415940  25.261584
Other                11.207970  12.107623
Registration/plates  10.294728  13.751868
Seat belt             2.573682   1.943199


**- Does Gender affects whose vehicle is searched ?**</br>


In [206]:
all = pd.DataFrame()
all['Male %'] = male.search_conducted.value_counts(normalize=True) * 100
all['Female %'] = female.search_conducted.value_counts(normalize=True) * 100
all['Total %'] = police_stops_ri.search_conducted.value_counts(normalize=True) * 100
print(all)

          Male %   Female %    Total %
False  95.457442  98.081938  96.178469
True    4.542558   1.918062   3.821531


In [205]:
police_stops_ri.search_conducted.mean() * 100 # Total Search Rate

3.82153092354627

In [203]:
male.search_conducted.mean() * 100 # Male Search Rate

4.542557598546892

In [207]:
female.search_conducted.mean() * 100  # Female Search Rate

1.9180617481282074

In [210]:
police_stops_ri.groupby('driver_gender').search_conducted.mean() * 100

driver_gender
F    1.918062
M    4.542558
Name: search_conducted, dtype: float64

###### Conclusion: Wow! Male drivers are searched more than twice as often as female drivers. Why might this be?

In [213]:
# Calculate the search rate for each combination of gender and violation
print(police_stops_ri.groupby([ 'violation', 'driver_gender']).search_conducted.mean() * 100)

violation            driver_gender
Equipment            F                 3.998401
                     M                 7.149644
Moving violation     F                 3.925746
                     M                 6.152419
Other                F                 4.101839
                     M                 4.619125
Registration/plates  F                 5.492424
                     M                10.880242
Seat belt            F                 1.730104
                     M                 3.511853
Speeding             F                 0.830883
                     M                 2.788541
Name: search_conducted, dtype: float64
